In [2]:
import duckdb

# Connect to existing DuckDB
conn = duckdb.connect("weather.duckdb")

# Define columns to extract
selected_columns = [
    "CALL_SIGN", "CIG", "DATE", "DEW", "ELEVATION", "GF1", "LATITUDE", "LONGITUDE",
    "NAME", "STATION", "TMP", "VIS", "WND"
]

# Create a new table `potsdam_filtered` from `combined_weather`
create_filtered_table_sql = f"""
    CREATE OR REPLACE TABLE combined_filtered AS
    SELECT {', '.join(selected_columns)}
    FROM combined_weather
    WHERE STATION IN (
        '72518699999', '72622394725', '72622894740',
        '74370014715', '99822399999', '99843599999'
    )
"""
conn.execute(create_filtered_table_sql)

# Show first few rows of the new filtered table
result = conn.execute("SELECT * FROM combined_filtered LIMIT 5").fetchdf()
result


,CALL_SIGN,CIG,DATE,DEW,ELEVATION,GF1,LATITUDE,LONGITUDE,NAME,STATION,TMP,VIS,WND
0,OGS,"06600,1,E,N",1977-05-02T00:00:00,"+9999,9",90.52,"07,99,1,99,9,99,9,99999,9,99,9,99,9",44.681854,-75.4655,"OGDENSBURG INTERNATIONAL, NY US",72518699999,"+0194,1","011200,1,N,1","200,1,N,0025,1"
1,OGS,"06600,1,9,N",1977-05-02T01:00:00,"+9999,9",90.52,"07,99,1,99,9,99,9,99999,9,99,9,99,9",44.681854,-75.4655,"OGDENSBURG INTERNATIONAL, NY US",72518699999,"+0183,1","016000,1,N,1","190,1,N,0030,1"
2,OGS,"22000,1,9,N",1977-05-03T11:00:00,"+9999,9",90.52,"00,99,1,99,9,99,9,99999,9,99,9,99,9",44.681854,-75.4655,"OGDENSBURG INTERNATIONAL, NY US",72518699999,"+0033,1","011200,1,N,1","340,1,N,0041,1"
3,OGS,"22000,1,9,N",1977-05-03T21:00:00,"+9999,9",90.52,"00,99,1,99,9,99,9,99999,9,99,9,99,9",44.681854,-75.4655,"OGDENSBURG INTERNATIONAL, NY US",72518699999,"+0128,1","011200,1,N,1","280,1,N,0030,1"
4,OGS,"02400,1,E,N",1977-05-05T11:00:00,"+9999,9",90.52,"08,99,1,99,9,99,9,99999,9,99,9,99,9",44.681854,-75.4655,"OGDENSBURG INTERNATIONAL, NY US",72518699999,"+0083,1","011200,1,N,1","999,1,C,0000,1"


In [4]:
print(result.head())

  CALL_SIGN          CIG                 DATE      DEW ELEVATION  \
0     OGS    06600,1,E,N  1977-05-02T00:00:00  +9999,9     90.52   
1     OGS    06600,1,9,N  1977-05-02T01:00:00  +9999,9     90.52   
2     OGS    22000,1,9,N  1977-05-03T11:00:00  +9999,9     90.52   
3     OGS    22000,1,9,N  1977-05-03T21:00:00  +9999,9     90.52   
4     OGS    02400,1,E,N  1977-05-05T11:00:00  +9999,9     90.52   

                                   GF1   LATITUDE LONGITUDE  \
0  07,99,1,99,9,99,9,99999,9,99,9,99,9  44.681854  -75.4655   
1  07,99,1,99,9,99,9,99999,9,99,9,99,9  44.681854  -75.4655   
2  00,99,1,99,9,99,9,99999,9,99,9,99,9  44.681854  -75.4655   
3  00,99,1,99,9,99,9,99999,9,99,9,99,9  44.681854  -75.4655   
4  08,99,1,99,9,99,9,99999,9,99,9,99,9  44.681854  -75.4655   

                              NAME      STATION      TMP           VIS  \
0  OGDENSBURG INTERNATIONAL, NY US  72518699999  +0194,1  011200,1,N,1   
1  OGDENSBURG INTERNATIONAL, NY US  72518699999  +0183,1  016000

In [ ]:
# SQL to create a cleaned table from potsdam_filtered
create_cleaned_table_sql = """
CREATE OR REPLACE TABLE combined_cleaned AS
SELECT
    CALL_SIGN,
    CIG,
    DATE,
    TRY_CAST(REPLACE(split_part(DEW, ',', 1), '+', '') AS DOUBLE) / 10 AS DEW_C,
    TRY_CAST(ELEVATION AS DOUBLE) AS ELEVATION_M,
    GF1,
    TRY_CAST(LATITUDE AS DOUBLE) AS LAT,
    TRY_CAST(LONGITUDE AS DOUBLE) AS LON,
    NAME,
    STATION,
    TRY_CAST(REPLACE(split_part(TMP, ',', 1), '+', '') AS DOUBLE) / 10 AS TEMP_C,
    TRY_CAST(REPLACE(split_part(VIS, ',', 1), '+', '') AS DOUBLE) AS VIS_M,
    WND
FROM combined_filtered
WHERE TMP IS NOT NULL AND TMP NOT LIKE '+9999%'
"""

# Execute the table creation
conn.execute(create_cleaned_table_sql)

# Preview first few cleaned records
cleaned_preview = conn.execute("SELECT * FROM combined_cleaned LIMIT 1000").fetchdf()
cleaned_preview


,CALL_SIGN,CIG,DATE,DEW_C,ELEVATION_M,GF1,LAT,LON,NAME,STATION,TEMP_C,VIS_M,WND
0,OGS,"06600,1,E,N",1977-05-02T00:00:00,999.9,90.52,"07,99,1,99,9,99,9,99999,9,99,9,99,9",44.681854,-75.4655,"OGDENSBURG INTERNATIONAL, NY US",72518699999,19.4,11200.0,"200,1,N,0025,1"
1,OGS,"06600,1,9,N",1977-05-02T01:00:00,999.9,90.52,"07,99,1,99,9,99,9,99999,9,99,9,99,9",44.681854,-75.4655,"OGDENSBURG INTERNATIONAL, NY US",72518699999,18.3,16000.0,"190,1,N,0030,1"
2,OGS,"22000,1,9,N",1977-05-03T11:00:00,999.9,90.52,"00,99,1,99,9,99,9,99999,9,99,9,99,9",44.681854,-75.4655,"OGDENSBURG INTERNATIONAL, NY US",72518699999,3.3,11200.0,"340,1,N,0041,1"
3,OGS,"22000,1,9,N",1977-05-03T21:00:00,999.9,90.52,"00,99,1,99,9,99,9,99999,9,99,9,99,9",44.681854,-75.4655,"OGDENSBURG INTERNATIONAL, NY US",72518699999,12.8,11200.0,"280,1,N,0030,1"
4,OGS,"02400,1,E,N",1977-05-05T11:00:00,999.9,90.52,"08,99,1,99,9,99,9,99999,9,99,9,99,9",44.681854,-75.4655,"OGDENSBURG INTERNATIONAL, NY US",72518699999,8.3,11200.0,"999,1,C,0000,1"


In [16]:
sql = """select * from combined_cleaned"""

result = conn.execute(sql).fetchdf()
print(result.head(20))

   CALL_SIGN          CIG                 DATE  DEW_C  ELEVATION_M  \
0      OGS    06600,1,E,N  1977-05-02T00:00:00  999.9        90.52   
1      OGS    06600,1,9,N  1977-05-02T01:00:00  999.9        90.52   
2      OGS    22000,1,9,N  1977-05-03T11:00:00  999.9        90.52   
3      OGS    22000,1,9,N  1977-05-03T21:00:00  999.9        90.52   
4      OGS    02400,1,E,N  1977-05-05T11:00:00  999.9        90.52   
5      OGS    22000,1,9,N  1977-05-11T16:00:00  999.9        90.52   
6      OGS    03000,1,9,N  1977-05-12T19:00:00  999.9        90.52   
7      OGS    22000,1,9,N  1977-05-21T12:00:00  999.9        90.52   
8      OGS    04500,1,9,N  1977-05-22T13:00:00  999.9        90.52   
9      OGS    03000,1,E,N  1977-05-22T15:00:00  999.9        90.52   
10     OGS    22000,1,9,N  1977-05-22T16:00:00  999.9        90.52   
11     OGS    22000,1,9,N  1977-05-27T00:00:00  999.9        90.52   
12     OGS    22000,1,9,N  1977-05-29T13:00:00  999.9        90.52   
13     OGS    22000,

In [28]:
from geopy.distance import geodesic

# Potsdam Coordinates
potsdam_coords = (44.676048, -74.9813)

# Step 1: Get STATION, LAT, LON from DuckDB directly
stations = conn.execute("""
    SELECT DISTINCT STATION, LAT, LON
    FROM combined_cleaned
    WHERE LAT IS NOT NULL AND LON IS NOT NULL
""").fetchall()

In [29]:
# Step 2: Compute normalized weights in Python
weights = {}
for station_id, lat, lon in stations:
    dist_km = geodesic(potsdam_coords, (lat, lon)).km
    weights[station_id] = 1 / max(dist_km**2, 0.0001)

# Normalize
total_weight = sum(weights.values())
normalized_weights = {sid: w / total_weight for sid, w in weights.items()}

In [30]:
# Step 3: Create a temporary weights table in DuckDB
conn.execute("DROP TABLE IF EXISTS station_weights")
conn.execute("CREATE TABLE station_weights (STATION VARCHAR, WEIGHT DOUBLE)")
for sid, weight in normalized_weights.items():
    conn.execute("INSERT INTO station_weights VALUES (?, ?)", [sid, weight])

In [41]:
# Step 4: Interpolation via SQL only (in-database)
interpolation_sql = """
CREATE OR REPLACE TABLE potsdam_weather_final AS
SELECT
    c.DATE,

    -- IDW-weighted average TEMP_C
    SUM(c.TEMP_C * w.WEIGHT) /
    NULLIF(SUM(w.WEIGHT) FILTER (WHERE c.TEMP_C < 999), 0) AS TEMP_C,

    -- Plain average DEW_C and VIS_M
    AVG(NULLIF(c.DEW_C, 999)) AS DEW_C,
    AVG(NULLIF(c.VIS_M, 999999)) AS VIS_M,

    -- IDW-weighted average ELEVATION
    SUM(c.ELEVATION_M * w.WEIGHT) /
    NULLIF(SUM(w.WEIGHT) FILTER (WHERE c.ELEVATION_M < 999), 0) AS ELEVATION_M,

    -- Static Potsdam metadata
    44.676048 AS LAT,
    -74.9813 AS LON,
    'POTSDAM, NY, USA' AS NAME

FROM combined_cleaned c
JOIN station_weights w ON c.STATION = w.STATION
GROUP BY c.DATE
ORDER BY c.DATE;
"""


In [42]:
# Step 5: Execute interpolation
conn.execute(interpolation_sql)

# Step 6: Preview final output
preview = conn.execute("SELECT * FROM potsdam_weather_final LIMIT 30").fetchdf()
print(preview)

                   DATE  TEMP_C  DEW_C    VIS_M  ELEVATION_M        LAT  \
0   1977-05-02T00:00:00    19.4  999.9  11200.0        90.52  44.676048   
1   1977-05-02T01:00:00    18.3  999.9  16000.0        90.52  44.676048   
2   1977-05-03T11:00:00     3.3  999.9  11200.0        90.52  44.676048   
3   1977-05-03T21:00:00    12.8  999.9  11200.0        90.52  44.676048   
4   1977-05-05T11:00:00     8.3  999.9  11200.0        90.52  44.676048   
5   1977-05-11T16:00:00    15.0  999.9  11200.0        90.52  44.676048   
6   1977-05-12T19:00:00    17.2  999.9  11200.0        90.52  44.676048   
7   1977-05-21T12:00:00    18.9  999.9   8000.0        90.52  44.676048   
8   1977-05-22T13:00:00    22.2  999.9   8000.0        90.52  44.676048   
9   1977-05-22T15:00:00    26.1  999.9   9600.0        90.52  44.676048   
10  1977-05-22T16:00:00    26.7  999.9  11200.0        90.52  44.676048   
11  1977-05-27T00:00:00    21.7  999.9  11200.0        90.52  44.676048   
12  1977-05-29T13:00:00  

In [44]:
sql = """select distinct dew_c from potsdam_weather_final"""

result = conn.execute(sql).fetchdf()
print(result)

           DEW_C
0      13.300000
1      20.300000
2       8.700000
3      11.200000
4      22.100000
...          ...
2093  -38.300000
2094    7.333333
2095    6.300000
2096  499.300000
2097  500.350000

[2098 rows x 1 columns]


In [46]:
sql = """select count(*) from potsdam_weather_final"""
result = conn.execute(sql).fetchdf()
print(result)

   count_star()
0       1168675


In [3]:
import duckdb

# Connect to DuckDB
con = duckdb.connect('weather.duckdb')

# Each query must be executed separately
con.execute("CREATE OR REPLACE TABLE temp_trend AS SELECT DATE, TEMP_C FROM potsdam_weather_final;")

con.execute("CREATE OR REPLACE TABLE vis_trend AS SELECT DATE, VIS_M FROM potsdam_weather_final;")

con.execute("CREATE OR REPLACE TABLE dew_cleaned AS SELECT DATE, DEW_C FROM potsdam_weather_final WHERE DEW_C != 999.9;")

con.execute("""
CREATE OR REPLACE TABLE daily_avg_temp AS 
SELECT DATE_TRUNC('day', CAST(DATE AS TIMESTAMP)) AS day, AVG(TEMP_C) AS avg_temp 
FROM potsdam_weather_final 
GROUP BY day;
""")

con.execute("""
CREATE OR REPLACE TABLE daily_min_max_temp AS 
SELECT DATE_TRUNC('day', CAST(DATE AS TIMESTAMP)) AS day, 
       MIN(TEMP_C) AS min_temp, 
       MAX(TEMP_C) AS max_temp 
FROM potsdam_weather_final 
GROUP BY day;
""")

con.execute("""
CREATE OR REPLACE TABLE temp_vs_vis AS 
SELECT TEMP_C, VIS_M 
FROM potsdam_weather_final 
WHERE TEMP_C IS NOT NULL AND VIS_M IS NOT NULL;
""")

con.execute("CREATE OR REPLACE TABLE temp_only AS SELECT TEMP_C FROM potsdam_weather_final;")
